In [19]:
import faiss
import os
import numpy as np
import pandas as pd
import scipy
from scipy.spatial.distance import cosine
import random
from sklearn.metrics import classification_report, accuracy_score
%matplotlib inline

In [20]:
from util import *

In [21]:
DATASET = 'data/LOCALHIST'
TRAIN_CSV = DATASET+'_df.csv'
TEST_VID_DIR = 'data/testvid'
FEATURE_EXTRACTOR = local_histogram_features

In [22]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

,video_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_890,x_891,x_892,x_893,x_894,x_895,x_896,x_897,x_898,x_899
0,../data/1943 - Victory Through Air Power.avi,0.033367,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../data/1943 - Victory Through Air Power.avi,2.035369,0.006034,0.000000,0.000000,0.0,0.0,0.0,0.067980,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,../data/1943 - Victory Through Air Power.avi,2.068735,0.014095,0.000397,0.000000,0.0,0.0,0.0,0.058366,0.000993,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,../data/1943 - Victory Through Air Power.avi,2.102102,0.014432,0.005248,0.000000,0.0,0.0,0.0,0.028427,0.039361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../data/1943 - Victory Through Air Power.avi,2.135469,0.003429,0.009715,0.000381,0.0,0.0,0.0,0.021717,0.044386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
class FaissVideoClassifier:
    def __init__(self, vectors, labels, treshold=0.3, ncells=0.01):
        """
        N - total amount of video frames
        K - amount of features
        vectors: N x K frame descriptor vectors
        labels: N x 1 frame labels
        """
        vectors = np.ascontiguousarray(vectors.astype('float32'))
        faiss.normalize_L2(vectors)
        self.vectors = np.ascontiguousarray(vectors)
        
        quantizer = faiss.IndexFlatL2(self.vectors.shape[1])  # the other index
        self.index = faiss.IndexIVFFlat(quantizer, self.vectors.shape[1], int(ncells*len(self.vectors)), faiss.METRIC_INNER_PRODUCT)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        self.labels = labels
        self.treshold = treshold
        
    def classify(self, vectors):
        """
        Majority vote
        """
        vectors = np.ascontiguousarray(vectors.astype('float32'))
        faiss.normalize_L2(vectors)
        vectors = np.ascontiguousarray(vectors)
        D, I = self.index.search(vectors, 1)
        min_indices, min_dists = I.flatten(), D.flatten()

        votes = self.labels[min_indices]
        miss_mask = abs(1 - min_dists) > self.treshold
        votes[miss_mask] = 'miss'
        
        moc = max([(list(votes).count(chr),chr) for chr in set(votes)])
        moc = moc[1]
        return moc, votes, min_dists

In [24]:
train_df.head()

,video_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_890,x_891,x_892,x_893,x_894,x_895,x_896,x_897,x_898,x_899
0,../data/1943 - Victory Through Air Power.avi,0.033367,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../data/1943 - Victory Through Air Power.avi,2.035369,0.006034,0.000000,0.000000,0.0,0.0,0.0,0.067980,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,../data/1943 - Victory Through Air Power.avi,2.068735,0.014095,0.000397,0.000000,0.0,0.0,0.0,0.058366,0.000993,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,../data/1943 - Victory Through Air Power.avi,2.102102,0.014432,0.005248,0.000000,0.0,0.0,0.0,0.028427,0.039361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../data/1943 - Victory Through Air Power.avi,2.135469,0.003429,0.009715,0.000381,0.0,0.0,0.0,0.021717,0.044386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
frame_times = train_df['frame_time']
labels = train_df.video_path.values
frame_vectors = train_df.drop(['frame_time', 'video_path'], axis=1).values
frame_vectors

array([[9.99999940e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.03374280e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.40951686e-02, 3.97046999e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.99999523e-01, 7.06214341e-04, 4.70809580e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.24734801e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [26]:
clf = FaissVideoClassifier(frame_vectors, labels, treshold=0.8)

In [27]:
test_videos = [os.path.join(TEST_VID_DIR, f) for f in os.listdir(TEST_VID_DIR)]
for fpath in test_videos:
    print(fpath)
    test_df = get_dataframe([fpath], feature_extractor=FEATURE_EXTRACTOR)   
    test_df_frame_vectors = test_df.drop(['frame_time', 'video_path'], axis=1).values
    predicted_label, votes, dists = clf.classify(test_df_frame_vectors)
    print('Classified as', predicted_label)
    print(votes, dists)

data/testvid/the_fun_and_fancy_tree_6.mp4
Done 1.0
Classified as ../data/1940 - Pinocchio.avi
['../data/1941 - The Reluctant Dragon.avi'
 '../data/1946 - Make Mine Music.avi'
 '../data/1941 - The Reluctant Dragon.avi' '../data/1940 - Pinocchio.avi'
 '../data/1942 - Saludos Amigos.m4v' '../data/1940 - Pinocchio.avi'
 '../data/1940 - Pinocchio.avi' '../data/1940 - Pinocchio.avi'
 '../data/1940 - Pinocchio.avi' '../data/1940 - Pinocchio.avi'
 '../data/1940 - Pinocchio.avi' '../data/1940 - Pinocchio.avi'] [0.8687181  0.8701353  0.8701413  0.86352825 0.8629971  0.86090106
 0.8614066  0.8603812  0.8639659  0.8715586  0.86744815 0.8798378 ]
data/testvid/the_fun_and_fancy_tree_4.mp4
Done 1.0
Classified as ../data/1940 - Pinocchio.avi
['../data/1946 - Make Mine Music.avi'
 '../data/1949 - The Adventures Of Ichabod And Mr. Toad.m4v'
 '../data/1946 - Make Mine Music.avi' '../data/1946 - Make Mine Music.avi'
 '../data/1948 - So Dear to My Heart.mp4' '../data/1940 - Pinocchio.avi'
 '../data/1940 - 